In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

## Auto-grad

In [2]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

In [3]:
y = w * x + b

In [4]:
y.backward()

In [5]:
print x.grad
print w.grad
print b.grad

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



## Basic autograd example 2

In [7]:
x = Variable(torch.randn(5, 3))
y = Variable(torch.randn(5, 2))
linear = nn.Linear(3, 2)

In [8]:
print 'w:', linear.weight
print 'b:', linear.bias

w: Parameter containing:
 0.5579  0.0533  0.5382
 0.5544  0.4830  0.2249
[torch.FloatTensor of size 2x3]

b: Parameter containing:
 0.0390
 0.1856
[torch.FloatTensor of size 2]



In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)
pred = linear(x)
print 'pred:', pred

pred: Variable containing:
-1.8514 -1.2267
-0.1659  0.0622
-0.4900 -0.3907
 1.9085  0.5578
 0.0063  0.0389
[torch.FloatTensor of size 5x2]



In [10]:
loss = criterion(pred, y)
print 'loss:', loss

loss: Variable containing:
 2.0893
[torch.FloatTensor of size 1]



In [11]:
loss.backward()
print 'dL/dw:', linear.weight.grad
print 'dL/db:', linear.bias.grad

dL/dw: Variable containing:
 0.4403 -0.4173  2.3348
 1.2912 -0.8032  1.1476
[torch.FloatTensor of size 2x3]

dL/db: Variable containing:
 0.7460
-0.1038
[torch.FloatTensor of size 2]



In [12]:
optimizer.step()

In [13]:
pred = linear(x)
loss = criterion(pred, y)
print 'loss after 1 step optimization:', loss.data[0]

loss after 1 step optimization: 1.99057543278


## Loading data

In [16]:
a = np.array([[1, 2], [3, 4]])
b = torch.from_numpy(a)   #np -> torch
c = b.numpy()             #torch -> np

In [17]:
train_dataset = dsets.CIFAR10(root='../data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

Extracting tar file
Done!


In [19]:
image, label = train_dataset[0]
print image.size()
print label

torch.Size([3, 32, 32])
6


In [20]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

In [21]:
data_iter = iter(train_loader)
images, labels = data_iter.next()

In [22]:
for images, labels in train_loader:
    pass

## Custom dataset

In [26]:
class CustomDataset():
    def __init__(self):
        # 1. Initialize file path or list of file names.
        pass
    def __getitem__(self, index):
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        return 0

custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

## Using pretrained model

In [27]:
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://s3.amazonaws.com/pytorch/models/resnet18-5c106cde.pth" to /Users/zal/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:07<00:00, 9543694.27it/s]


In [28]:
for param in resnet.parameters():
    param.requires_grad = False

In [29]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

In [31]:
images = Variable(torch.randn(10, 3, 256, 256))
outputs = resnet(images)
print outputs.size()
print outputs

torch.Size([10, 100])
Variable containing:
-0.0268  0.5486 -0.2649  ...  -0.7075  0.3512  0.2783
 0.2035  0.9978 -0.3668  ...  -0.6232  0.2231  0.3093
 0.1271  0.5256 -0.0925  ...   0.0782 -0.2478  0.4704
          ...             ⋱             ...          
 0.3371  0.8388 -0.4171  ...  -0.9723  0.3796  0.2666
 0.3764  1.0292 -0.8678  ...  -0.0171 -0.2020  0.6846
-0.4773  0.2810  0.3216  ...  -0.8836  0.7359  0.4897
[torch.FloatTensor of size 10x100]



Variable containing:
-0.0268
 0.5486
-0.2649
 0.4104
 0.5322
 1.1841
-0.5812
-0.0493
 0.0542
-0.3008
-0.4043
-0.3789
 1.0509
-0.8859
-0.5165
-0.4250
-1.3922
 0.7892
-1.0385
 0.1172
-1.2449
 0.2854
 0.1718
-0.2114
-0.1537
-0.7267
 0.0537
 0.4967
-0.6089
 0.2851
-0.3053
-0.3085
 1.1602
-0.2257
 0.4405
 0.8428
-1.1567
 0.0100
 0.8346
-0.5473
-0.2915
 0.8987
 0.1276
-1.4882
-0.4107
 0.1422
-0.3522
-0.7499
 0.5616
-1.0327
 0.1323
-0.2244
-0.2546
-0.3500
-0.4735
-0.0302
-0.1849
-0.8251
 0.5751
-0.3224
-1.4644
 0.4885
-0.2437
 0.7424
-0.3115
 1.0801
 0.0276
 0.2142
-0.1504
 0.4341
 1.3530
 1.0293
 0.0300
 0.2730
 0.8081
-0.3452
 0.2966
 0.5419
-1.0668
-0.1672
-0.1610
-0.2529
 0.4704
-0.3122
-0.6760
 0.2370
 0.0577
-0.7685
 0.6545
-1.0316
-0.1468
 0.8644
-0.0980
 0.8381
-0.6197
-0.2307
 0.4433
-0.7075
 0.3512
 0.2783
[torch.FloatTensor of size 100]



## Save and load

In [33]:
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

In [34]:
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))